# Cálculo de métricas de los cuestionarios

El objetivo de este notebook será calcular con los datos obtenidos de las entregas de cuestionarios por parte de los alumnos las siguientes métricas:

- Número de intentos totales a lo largo del curso
- Número de intentos por mes
- Número máximo de semanas sin ningún intento
- Promedio de intentos por cuestionario

## Configuración y carga de datos